In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [3]:
# Example: load a DSS dataset as a Pandas dataframe
mydataset = dataiku.Dataset("LWINdatabase")
mydataset_df = mydataset.get_dataframe()

In [4]:
mydataset_df

,LWIN,STATUS,DISPLAY_NAME,PRODUCER_TITLE,PRODUCER_NAME,WINE,COUNTRY,REGION,SUB_REGION,SITE,PARCEL,COLOUR,TYPE,SUB_TYPE,DESIGNATION,CLASSIFICATION,VINTAGE_CONFIG,FIRST_VINTAGE,FINAL_VINTAGE,DATE_ADDED,DATE_UPDATED,REFERENCE
0,1000001,Live,"Schieferkopf, Lieu Dit Buehl Riesling",NaN,Schieferkopf,Lieu Dit Buehl Riesling,France,Alsace,NaN,NaN,NaN,White,Wine,Still,AOP,NaN,sequential,NaN,NaN,NaT,NaT,NaN
1,1000014,Live,"Schieferkopf, Riesling Grand Cru, Kastelberg",NaN,Schieferkopf,Riesling,France,Alsace,Kastelberg,NaN,NaN,White,Wine,Still,AOP,Grand Cru,sequential,NaN,NaN,NaT,NaT,NaN
2,1000027,Live,"Schieferkopf, Pinot Gris",NaN,Schieferkopf,Pinot Gris,France,Alsace,NaN,NaN,NaN,White,Wine,Still,AOP,NaN,sequential,NaN,NaN,NaT,NaT,NaN
3,1000030,Live,"Schieferkopf, Riesling Grand Cru, Altenberg de...",NaN,Schieferkopf,Riesling,France,Alsace,Altenberg de Bergheim,NaN,NaN,White,Wine,Still,AOP,Grand Cru,sequential,NaN,NaN,NaT,NaT,NaN
4,1000043,Live,"Schieferkopf, Sylvaner",NaN,Schieferkopf,Sylvaner,France,Alsace,NaN,NaN,NaN,White,Wine,Still,AOP,NaN,sequential,NaN,NaN,NaT,NaT,NaN
5,1000056,Live,"Schieferkopf, Weibelsberg Riesling",NaN,Schieferkopf,Weibelsberg Riesling,France,Alsace,NaN,NaN,NaN,White,Wine,Still,AOP,NaN,sequential,NaN,NaN,NaT,NaT,NaN
6,1000069,Live,"Hugel, Riesling Jubilee Grand Cru, Altenberg d...",NaN,Hugel,Riesling Jubilee,France,Alsace,Altenberg de Bergheim,NaN,NaN,White,Wine,Still,AOP,Grand Cru,sequential,NaN,NaN,NaT,NaT,NaN
7,1000072,Live,"Hugel, Riesling Selection de Grains Nobles Gra...",NaN,Hugel,Riesling Selection de Grains Nobles,France,Alsace,Altenberg de Bergheim,NaN,NaN,White,Wine,Still,AOP,Grand Cru,sequential,NaN,NaN,NaT,NaT,NaN
8,1000085,Live,"Hugel, Riesling Vendanges Tardives Grand Cru, ...",NaN,Hugel,Riesling Vendanges Tardives,France,Alsace,Altenberg de Bergheim,NaN,NaN,White,Wine,Still,AOP,Grand Cru,sequential,NaN,NaN,NaT,NaT,NaN
9,1000098,Live,"Hugel, Tokay Pinot Gris Jubilee",NaN,Hugel,Tokay Pinot Gris Jubilee,France,Alsace,NaN,NaN,NaN,White,Wine,Still,AOP,NaN,sequential,NaN,NaN,NaT,NaT,NaN


In [5]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_wine_like(self, person_name, wine_name):
        with self.driver.session(database='wine') as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_wine_like, person_name, wine_name)
            for row in result:
                print("Created like between: {p}, {w}".format(p=row['p'], w=row['w']))

    @staticmethod
    def _create_wine_like(tx, person_name, wine_name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MERGE (p:Person { name: $person_name }) "
            "MATCH (w:Wine { name: $wine_name }) "
            "MERGE (p)-[:LIKES]->(w) "
            "RETURN p, w"
        )
        result = tx.run(query, person_name=person_name, wine_name=wine_name)
        try:
            return [{"p": row["p"]["name"], "w": row["w"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self, person_name):
        with self.driver.session(database='wine') as session:
            result = session.read_transaction(self._find_and_return_person, person_name)
            for row in result:
                print("Found person: {row}".format(row=row))

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:Person) "
            "WHERE p.name = $person_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, person_name=person_name)
        return [row["name"] for row in result]


if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+ssc://35.181.44.209/wine"
    user = "neo4j"
    password = "neo4jdss"
    app = App(uri, user, password)
    app.create_friendship("Alice", "David")
    app.find_person("Alice")
    app.close()

Created friendship between: Alice, David
Found person: Alice
